# ADA Project Milestone P3

## <span style="color: green;">Import libraries and set paths </span>

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode as ud
from tqdm import tqdm
from collections import Counter
from datetime import datetime
%matplotlib inline

from preprocessing_functions import *


### Set the global variables for the paths to the data folder

In [2]:
# add your data folder into the 'code' folder, make sure it is in the .gitignore file

CODE_DIR = os.getcwd()
DATA_DIR = os.path.join(CODE_DIR, 'data')

print('your data directory:')
DATA_DIR

your data directory:


'c:\\Users\\fulci\\git\\ada_23\\ada-2023-project-adarescueteam\\code\\data'

In [3]:
# set path for BeerAdvocate data
DATA_BeerAdvocate = os.path.join(DATA_DIR, "BeerAdvocate.tar")

# set path for RateBeer data
DATA_RateBeer = os.path.join(DATA_DIR, "RateBeer.tar")

# set path for MatchedBeer data
DATA_MatchedBeers = os.path.join(DATA_DIR, "matched_beer_data.tar")

## <span style="color: green;">Load cleaned data </span>

## RaterBeer (RB) Data

#### beers, breweries, users csv's

In [4]:
# read in the BeerAdvocate files
BA_beers = pd.read_csv(os.path.join(DATA_BeerAdvocate, "beers.csv"))
BA_breweries = pd.read_csv(os.path.join(DATA_BeerAdvocate, "breweries.csv"))
BA_users = pd.read_csv(os.path.join(DATA_BeerAdvocate, "users.csv"), 
                       converters={"joined": convert_timestamp})

#### ratings pickle

In [5]:
# read RB_ratings from pickle file
with open(os.path.join(DATA_RateBeer, "RB_ratings.pkl"), 'rb') as f:
    RB_ratings = pickle.load(f)


## BeerAdvocate (BA) Data

#### beers, breweries, users csv's

In [6]:
# read in the RateBeer files
RB_beers = pd.read_csv(os.path.join(DATA_RateBeer, "beers.csv"))
RB_breweries = pd.read_csv(os.path.join(DATA_RateBeer, "breweries.csv"))
RB_users = pd.read_csv(os.path.join(DATA_RateBeer, "users.csv"),
                       converters={"joined": convert_timestamp})

#### ratings pickle

In [7]:
# read BA_ratings from pickle file
with open(os.path.join(DATA_BeerAdvocate, "BA_ratings.pkl"), 'rb') as f:
    BA_ratings = pickle.load(f)

## Glutenfree (gf) Beer Data

In [8]:
# read in the glutenfree beers
rb_gf_ratings=pd.read_csv(os.path.join(DATA_RateBeer, 'rb_gf_ratings.csv'), low_memory=False, encoding='utf-8')
ba_gf_ratings=pd.read_csv(os.path.join(DATA_BeerAdvocate, 'ba_gf_ratings.csv'), low_memory=False, encoding='utf-8')
rb_gf_ratings.shape, ba_gf_ratings.shape

((3770, 18), (2397, 19))

In [9]:
# add the review columns to the rb_gf_ratings
rb_gf_ratings['review'] = True

In [10]:
# merge the two datasets
gf_ratings = pd.concat([rb_gf_ratings, ba_gf_ratings])
gf_ratings.shape

(6167, 19)

In [11]:
# check duplicates
gf_ratings.duplicated().sum()

0

## <span style="color: green;">Filter data for matching </span>

### Filter by style

In this section, we want to match the glutenfree (gf) beers by style with the conventional beers. For this we can remove all the style in the BA or RB dataset, which is not represented in the gf beer dataset.

Following this, we merge the two _style datasets and check for duplicates.

In [12]:
#Keep only beer-types present in gluten-free beers

gf_beers=gf_ratings['beer_name'].unique()

for i in range(len(gf_beers)): #Somehow there is a space at beginning of beer_names
    gf_beers[i]=gf_beers[i][1:]

BA_gf_beertypes = BA_beers[BA_beers['beer_name'].isin(gf_beers)]['style'].unique()
RB_gf_beertypes = RB_beers[RB_beers['beer_name'].isin(gf_beers)]['style'].unique()

In [13]:
gf_beertypes = np.concatenate((BA_gf_beertypes,RB_gf_beertypes))
gf_beertypes=pd.DataFrame(gf_beertypes,columns=['style'])
gf_beertypes=gf_beertypes['style'].unique()

In [14]:
# represent only the styles in the full dataset that are also represented in the glutenfree dataset
# Here we do the comparison by first extracting the beer_id with the beer style from the datasets, as there is issues comparing directly the array gf_beertypes

# for BeerAdvocate 
BA_beers[BA_beers['style'].isin(gf_beertypes)]  # 169793 rows
BA_beerID_style = BA_beers[BA_beers['style'].isin(gf_beertypes)][['beer_id', 'style']]
BA_ratings_style = BA_ratings[BA_ratings['beer_id'].isin(BA_beerID_style['beer_id'])]

# and the same for Ratebeer
RB_beers[RB_beers['style'].isin(gf_beertypes)]
RB_beerID_style = RB_beers[RB_beers['style'].isin(gf_beertypes)][['beer_id', 'style']]
RB_ratings_style = RB_ratings[RB_ratings['beer_id'].isin(RB_beerID_style['beer_id'])]

In [15]:
# merge this two dataframes
    # beforehands, we add a review column to avoid nan conflicts later
RB_ratings_style['review'] = True
ratings_style = pd.concat([BA_ratings_style, RB_ratings_style])
ratings_style.shape 

C:\Users\fulci\AppData\Local\Temp\ipykernel_14488\130779583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RB_ratings_style['review'] = True


(9371934, 17)

In [16]:
# check for duplicates
ratings_style.duplicated().sum()

12

In [17]:
# remove all duplicates and check again
ratings_style = ratings_style[~ratings_style.duplicated(keep='first')]
ratings_style.duplicated().sum()

0

We want to make sure that the glutenfree beers are not contained in the full data, especially because not subtracting the gf data would produce duplicates in the analysis and thus unwanted perfect matchings between the same samples.

In [18]:
# merge the two dfs and only keep the rows that are present in ratings_style but not in gf_ratings
merged_ratings = pd.merge(ratings_style, gf_ratings, indicator=True, how='outer')
ratings_style = merged_ratings[merged_ratings['_merge'] == 'left_only'].drop(columns=['_merge', 'location', 'year'])
ratings_style

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,review
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,2015-08-20 10:00:00,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88,"From a bottle, pours a piss yellow color with...",True
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2009-02-20 11:00:00,StJamesGate,stjamesgate.163714,3.0,3.5,3.5,4.0,3.5,3.67,Pours pale copper with a thin head that quick...,True
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2006-03-13 11:00:00,mdagnew,mdagnew.19527,4.0,3.5,3.5,4.0,3.5,3.73,"500ml Bottle bought from The Vintage, Antrim....",True
3,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2004-12-01 11:00:00,helloloser12345,helloloser12345.10867,4.0,3.5,4.0,4.0,4.5,3.98,Serving: 500ml brown bottlePour: Good head wi...,True
4,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,2004-08-30 10:00:00,cypressbob,cypressbob.3708,4.0,4.0,4.0,4.0,4.0,4.00,"500ml bottlePours with a light, slightly hazy...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9371917,Svejk Blonde,220897,Svejk Beer Garden,17155,Pale Lager,nan,2014-09-18 10:00:00,Travlr,83882,3,6,2,5,10,2.60,"Draft at the source. Clear golden color, fluf...",True
9371918,Svejk Blonde,220897,Svejk Beer Garden,17155,Pale Lager,nan,2013-12-01 11:00:00,TBone,10233,2,5,2,6,10,2.50,"Tap @brewpub, TiraneClear golden color, good ...",True
9371919,Svejk Dark,220898,Svejk Beer Garden,17155,Dunkel/Tmavý,nan,2014-11-04 11:00:00,Rob_D_UK,257161,3,4,2,5,9,2.30,In their beer garden after a walking tour aro...,True
9371920,Svejk Dark,220898,Svejk Beer Garden,17155,Dunkel/Tmavý,nan,2014-09-16 10:00:00,Travlr,83882,3,5,1,4,6,1.90,"Draft at the source. Hazy maroon color, tan h...",True


Now let's prepare the dataset for the third question which looks at the reviews, we filter out the samples that have no review and create **reviews_style**

In [19]:
(ratings_style['review']!=True).sum()

4810208

In [20]:
reviews_style = ratings_style[(ratings_style['review'] == True)]
reviews_style.shape

(4557944, 17)

Finally, we only want to focus on the reviews that are either english or french, thus we remove all languages that we are not interested in.

In [51]:
# split the dataframe
parts = 7
length = (len(reviews_style)//parts)+1

reviews_style_1 = reviews_style.iloc[:length,:]
reviews_style_2 = reviews_style.iloc[length:2 * length,:]
reviews_style_3 = reviews_style.iloc[2 * length:3 * length,:]
reviews_style_3 = reviews_style.iloc[2 * length:3 * length, :]
reviews_style_4 = reviews_style.iloc[3 * length:4 * length, :]
reviews_style_5 = reviews_style.iloc[4 * length:5 * length, :]
reviews_style_6 = reviews_style.iloc[5 * length:6 * length, :]
reviews_style_7 = reviews_style.iloc[6 * length:, :]


In [63]:
# the more efficient way..
parts = 7
length = (len(reviews_style) // parts) + 1
split_reviews = []
for i in range(parts):
    start_idx = i * length
    end_idx = min((i + 1) * length, len(reviews_style))
    split_reviews.append(reviews_style.iloc[start_idx:end_idx, :])
reviews_style_1, reviews_style_2, reviews_style_3, reviews_style_4, reviews_style_5, reviews_style_6, reviews_style_7 = split_reviews #call parts either with split_reviews[0] or reviews_style_xy

In [52]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [53]:
reviews_languages = []

for row in tqdm(range(len(reviews_style_1))):
    try:
        doc = nlp(reviews_style_1['text'].iloc[row])
        reviews_languages.append(doc._.language['language'])
    except Exception as e:
        reviews_languages.append(np.nan)

  0%|          | 0/651135 [00:00<?, ?it/s]

100%|██████████| 651135/651135 [2:11:58<00:00, 82.23it/s]   


In [22]:
from multiprocessing import Pool, cpu_count

def process_text(row):
    try:
        doc = nlp(reviews_style['text'].iloc[row])
        return doc._.language['language']
    except Exception as e:
        return np.nan

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

reviews_languages = []
chunk_size = 10000  # You can adjust this chunk size based on memory constraints
chunks_list = list(chunks(range(len(reviews_style)), chunk_size))

def process_chunk(chunk):
    return [process_text(row) for row in chunk]

with Pool(cpu_count() - 1) as p:
    results = p.map(process_chunk, chunks_list)

for result in results:
    reviews_languages.extend(result)

In [60]:
reviews_style_1

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,review
4810208,33 Export (Gabon),410549,Sobraga,3198,Pale Lager,5.0,2016-04-26 10:00:00,Manslow,175852,2,4,2,4,8,2.0,"Puszka 0,33l dzięki Christoph . Kolor jasnozł...",True
4810209,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,2017-02-17 11:00:00,MAGICuenca91,442761,2,3,2,4,8,1.9,Cerveza pale lager gabonesa. Más floja que la...,True
4810210,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,2016-06-24 10:00:00,Sibarh,288889,3,3,2,3,5,1.6,"Kolor- złoty, klarowny. Piana - drobna, średn...",True
4810211,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,2016-01-01 11:00:00,fombe89,250510,4,3,1,2,5,1.5,"Botella, de Gabón regalo familiar.31/01/2015C...",True
4810212,Castel Beer (Gabon),105273,Sobraga,3198,Pale Lager,5.2,2015-10-23 10:00:00,kevnic2008,122778,2,4,2,4,7,1.9,Many thanks for this beer to Erzengel. Pours ...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5462570,Duckstein Rotblondes Weizen,57146,Mecklenburgische Brauerei Lübz &#40;Carlsberg...,2513,German Hefeweizen,5.7,2009-06-24 10:00:00,larsga,15206,4,6,4,7,14,3.5,Biggish white lasting head. Hazy opaque reddi...,True
5462571,Duckstein Rotblondes Weizen,57146,Mecklenburgische Brauerei Lübz &#40;Carlsberg...,2513,German Hefeweizen,5.7,2009-06-12 10:00:00,ekstedt,20248,4,6,2,6,12,3.0,Sample (10 cl) at Gothenburg Beer Festival 20...,True
5462572,Duckstein Rotblondes Weizen,57146,Mecklenburgische Brauerei Lübz &#40;Carlsberg...,2513,German Hefeweizen,5.7,2009-06-11 10:00:00,gnoff,32615,4,5,2,5,9,2.5,"Bottle at ""City Pub"" Göteborg during the Goth...",True
5462573,Duckstein Rotblondes Weizen,57146,Mecklenburgische Brauerei Lübz &#40;Carlsberg...,2513,German Hefeweizen,5.7,2009-06-05 10:00:00,anders37,18419,3,5,3,5,13,2.9,Bottle. Pours a slightly hazy reddish amber c...,True


In [70]:
reviews_style_1['lang'] = reviews_languages
reviews_style_1[reviews_style_1['lang'].isna()]

C:\Users\fulci\AppData\Local\Temp\ipykernel_14488\4269624641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_style_1['lang'] = reviews_languages


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,review,lang
4813935,Tiger Beer,3126,Singapore Brewery (Asia Pacific Breweries-Hei...,547,Pale Lager,5.0,2004-07-31 10:00:00,miko,12525,5,8,4,8,16,4.1,NaN,True,NaN
4827336,Star Lager (Nigeria),12804,Nigerian Breweries PLC (Heineken),2347,Pale Lager,5.1,2012-07-30 10:00:00,jamesie1857,166482,3,5,2,6,10,2.6,NaN,True,NaN
4834228,Old Bohemian Lager,12381,Pivovara Daruvar,1616,Pale Lager,5.0,2008-05-26 10:00:00,haf,11751,2,4,2,4,12,2.4,NaN,True,NaN
4852823,Corona Extra,742,Grupo Modelo (Corona),119,Pale Lager,4.5,2004-05-21 10:00:00,miko,12525,3,5,2,7,13,3.0,NaN,True,NaN
4854320,Corona Extra,742,Grupo Modelo (Corona),119,Pale Lager,4.5,2012-11-20 11:00:00,peteysmu,121478,3,10,3,6,12,3.4,NaN,True,NaN
4857523,Pacifico Clara,746,Grupo Modelo (Corona),119,Pale Lager,4.5,2003-03-29 11:00:00,Crash,5963,3,5,3,7,13,3.1,NaN,True,NaN
4902654,Dieu du Ciel / Hill Farmstead Friendship and ...,176057,Dieu du Ciel,364,India Pale Ale (IPA),6.0,2012-10-31 11:00:00,fregolo2,225200,4,8,4,9,19,4.4,NaN,True,NaN
4903082,Dieu du Ciel / The Alchemist Moralité,176058,Dieu du Ciel,364,India Pale Ale (IPA),6.9,2012-10-31 11:00:00,fregolo2,225200,4,8,4,8,16,4.0,NaN,True,NaN
4903277,Dieu du Ciel 13e Étage,154294,Dieu du Ciel,364,Spice/Herb/Vegetable,4.6,2012-10-31 11:00:00,fregolo2,225200,5,9,5,9,19,4.7,NaN,True,NaN
4903315,Dieu du Ciel Amère Khadir,96487,Dieu du Ciel,364,American Pale Ale,5.5,2012-10-31 11:00:00,fregolo2,225200,4,7,3,8,16,3.8,NaN,True,NaN


should have been filtered out... will sort out at the end

In [71]:
# save as pickle
with open(os.path.join(DATA_DIR, "reviews_style_1.pkl"), 'wb') as f:
    pickle.dump(reviews_style_1, f)

## <span style="color: green;">Merge the RB and BA dataframe after filtering </span>

shrink down the datasets before matching non-glutenfree reviews with glutenfree reviews, takes too long to work else

and not to forget: eliminate duplicates, because most likely as we dont consider the Matched_beer dataset => done


In [11]:
# add review columns to RB_ratings before joining them. all entries have reviews
RB_ratings['review']=True

In [17]:
frames = [BA_ratings, RB_ratings]
RB_BA_ratings = pd.concat(frames)
RB_BA_ratings.shape
# takes 3s

(15515106, 17)

In [13]:
RB_BA_ratings.duplicated().sum()
# takes 35s

18

In [18]:
RB_BA_ratings = RB_BA_ratings[~RB_BA_ratings.duplicated()]
# takes 35s

In [19]:
RB_BA_ratings.duplicated().sum()
# takes again 35s -> this is why we need to shrink down the data first

0

## <span style="color: green;">Temporal analyis</span>

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch 

## <span style="color: green;">Language processing</span>

suggest that we write functions / pipeline in an outside script and in here just leave the important things

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch 

-> dont forget to do the analysis on the whole rb_ba_gf dataset!

### Language identification

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch , need to see what to keep though

### Adjective and adverbs extraction

## <span style="color: lightgreen;">correlation of the reviews with the ratings</span> TBD
